In [61]:
import numpy as np
import matplotlib.pyplot as plt

#function , constraints
def obj_f(x):
    f=x[0][0]**2+(x[1][0]-3)**2
    return f

def obj_df(x):
    df=np.array([[2*x[0][0],2*(x[1][0]-3)]])
    return df

def cons_g(x):
    g=np.array([[x[1][0]**2-2*x[0][0]],
                  [(x[1][0]-1)**2+5*x[0][0]-15]])
    return g

def cons_dg(x):
    dg=np.array([[-2,2*x[1][0]],
                   [5,2*(x[1][0]-1)]])
    return dg

#BFGS
def BFGS(W,x,a,s,mu):
    yk=(obj_df(x) +np.matmul(mu.T,cons_dg(x)))-(obj_df(x-a*s)+np.matmul(mu.T,cons_dg(x-a*s)))
    a_s=a*s
    if np.matmul(a_s.T,yk.T)>=0.2*np.matmul(np.matmul(a_s.T,W),a_s):
        theta=1
    else:
        theta=0.8*np.matmul(np.matmul(a_s.T,W),a_s)/(np.matmul(np.matmul(a_s.T,W),a_s)-np.matmul(a_s.T,yk.T))
    gk=theta*yk.T+(1-theta)*np.matmul(W,a_s)
    W_new=W+np.matmul(gk,gk.T)/np.matmul(gk.T,s)-np.matmul(np.matmul(W,s),np.matmul(s.T,W))/np.matmul(np.matmul(s.T,W),s)
    return W_new

                
#Armijo Line Search to find the step size.
def line_search(x, s, mu, w_old, k):
    t=0.1    #factor(0.01.0.3)
    a=1      #max step size
    bac=0.8   #factor for backtracking
    
 # Calculate weights(w) in the merit function
    w=np.zeros((2, 1))
    w[0]=max(abs(mu[0]),0.5*(w_old[0]+abs(mu[0])))
    w[1]=max(abs(mu[1]),0.5*(w_old[1]+abs(mu[1])))
    sig=(np.sign(w)+1)/2
    com=np.zeros((2, 1))
    com[0]=max(0,cons_g(x)[0,:])
    com[1]=max(0,cons_g(x)[1,:])
    com1=np.zeros((2, 1))
    com1[0]=max(0,cons_g(x+a*s)[0,:])
    com1[1]=max(0,cons_g(x+a*s)[1,:])   
    def phi(x,w,sig,com,a,t,s):
        phii = obj_f(x) + np.matmul(w.T,com)+a*t*s* (obj_df(x)+np.matmul(w.T,sig))                                          
        return phii   
    def phinn(x,w,a,s,com1):
        phin=obj_f(x+a*s)+np.matmul(w.T,com1)
        return phin
    phiiend=phi(x,w,sig,com,a,t,s)[:,-1]
    phinend=phinn(x,w,a,s,com1)[:,-1] 
    while phiiend.any() <  phinend.any():
        a= bac*a
    return a, w  
                 

##SQP
def sqp(x, W):
    A0=cons_dg(x)
    b0=cons_g(x)
    mu0=np.zeros((b0.shape[0], 1))
    mu=[]
    active=[]
    while True:
        if len(active)==0:
            ww=W
            smu=np.matmul(np.linalg.inv(ww),-obj_df(x).T)
            s=smu[:2, :]
            mu=[]
        if len(active)!=0:
            if len(active)==1:
                A=A0[active[0],:].reshape(1,-1)  #the unspecified value
                b=b0[active[0],:]
            if len(active)==2:
                A=A0
                b=b0               
            ww=np.vstack((np.hstack((W, A.T)),
                                np.hstack((A,np.zeros((A.shape[0],A.shape[0]))))))
                       smu=np.matmul(np.linalg.inv(ww),np.vstack((-obj_df(x).T,-b)))
            s=smu[:2,:]
            mu=smu[2:,:]
            if len(mu)==1:
                mu0[0]=smu[2:3,:]
            if len(mu)==2:
                mu0[0]=smu[2:3,:]
                mu0[1]=smu[3:,:]
        sqp_cons=np.round((np.matmul(A0,s.reshape(-1,1))+b0))
        # Check mu values and set mucheck to 1 when they make sense
        mucheck=0
        if len(mu)==0:
            mucheck=1
        elif min(mu)>0:
            mucheck=1
        else:
            id_mu=np.argmin(np.array(mu))
            #REMOVE the most negaitve mu from acitve set
            mu.remove(min(mu))  
            active.pop(id_mu)
        if np.max(sqp_cons)<=0:
            if mucheck==1:
                return s,mu0 
        else:
            num=np.argmax(sqp_cons)
            active.append(num)
            active=np.unique(np.array(active)).tolist()
    return None


x0=np.array([[1.],[1.]])
x=np.array([[1.],[1.]])
W=np.eye(x.shape[0])
mu_old=np.zeros((x.shape[0], 1))
eps=1e-3 
k=0

ky_norm=np.linalg.norm(obj_df(x) + np.matmul(mu_old.T, cons_dg(x)))
w_old=np.zeros((2, 1))
ans1=[]
ans2=[]
ans1.append(x[0][0])
ans2.append(x[1][0])

while ky_norm > eps:
    s,mu_new = sqp(x, W)
    a,w_new = line_search(x, s, mu_old, w_old, k)
    w_old=w_new
    x+=a*s
    W=BFGS(W,x,a,s,mu_new)
    k+=1
    ky_norm=np.linalg.norm(obj_df(x)+np.matmul(mu_new.T,cons_dg(x)))
    mu_old=mu_new
    ans1.append(x[0][0])
    ans2.append(x[1][0])

print('Solution for this problem is: x=({},{})'.format(ans1[-1],ans2[-1]))

Solution for this problem is: x=(1.0604169033539967,1.4563356389457214)
